# Kernel Methods and SVMs

First install and import needed packages and libraries

In [ ]:
# !pip install numpy scipy pandas matplotlib scikit-learn missingno

In [ ]:
import numpy as np
import pandas as pd

We're going to use the [Iranian Churn Dataset](https://archive.ics.uci.edu/ml/datasets/Iranian+Churn+Dataset) from [Kaggle](https://www.kaggle.com/datasets/royjafari/customer-churn). 

The dataset was created with real world data of an Iranian telecom company. It provides an information about whether the company's client left them or not.

Such dataset can be used to help improving client's satisfaction and maximizing company's earnings.

The dataset contains information about costs of False Positive and False Negative classification for each client.

In [ ]:
import matplotlib.pyplot as plt

churn_df = pd.read_csv("customer_churn_data.csv")
churn_df.head()

In [ ]:
y = churn_df.pop("Churn")
fn_cost = churn_df.pop("FN")
fp_cost = churn_df.pop("FP")

In [ ]:
ax = y.value_counts()[[0, 1]].plot.bar(
    x="class", y="Number of clients", rot=0, title="Class distribution"
)
ax.set(xlabel="Class", ylabel="Number of clients")

Let's split and scale our data

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

(
    X_train,
    X_test,
    y_train,
    y_test,
    fp_cost_train,
    fp_cost_test,
    fn_cost_train,
    fn_cost_test,
) = train_test_split(
    churn_df, y, fp_cost, fn_cost, test_size=0.25, random_state=0, stratify=y
)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

Let's create a linear regression to use as our baseline for further comparison

In [ ]:
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegressionCV

logreg_cv_l2 = LogisticRegressionCV(
    Cs=100, cv=5, scoring="f1", class_weight="balanced", random_state=0, n_jobs=-1
)
logreg_cv_l2.fit(X_train, y_train)

y_pred = logreg_cv_l2.predict(X_test)
print(f"F1-score: {100 * f1_score(y_test, y_pred):.2f}%")

Now let's train a linear SVM classifier and see how it compares with linear regression

In [ ]:
from time import time
from sklearn.model_selection import GridSearchCV
from sklearn.svm import LinearSVC

clf = LinearSVC(
    loss="hinge", max_iter=10000, random_state=0, class_weight="balanced", dual="auto"
)

param_grid = {"C": np.power(10.0, np.arange(-2, 2, 4 / 100))}

cv = GridSearchCV(estimator=clf, param_grid=param_grid, scoring="f1", cv=5, n_jobs=-1)

start_time = time()
cv.fit(X_train, y_train)
end_time = time()

prediction_time = end_time - start_time

print("training time: ", prediction_time, "s")
print("best C: ", cv.best_params_["C"])

y_pred = cv.predict(X_test)
print(f"F1-score: {100 * f1_score(y_test, y_pred):.2f}%")

Both baseline score and SVM score are not great. Let's see if we can improve the score with Kernel SVM

In [ ]:
from sklearn.svm import SVC

clf_kernel_svc = SVC(cache_size=512, class_weight="balanced", random_state=0)
start_time = time()
clf_kernel_svc.fit(X_train, y_train)
end_time = time()
training_time = end_time - start_time

y_pred = clf_kernel_svc.predict(X_test)

print(f"F1-score: {100 * f1_score(y_test, y_pred):.2f}%")
print(f"Training time: {training_time} s")

The improvement is noticeable but still not as great as we would like.

Maybe tuning the hyperparameters will help? We'll try a few different kernels

In [ ]:
clf = SVC(cache_size=512, class_weight="balanced", random_state=0)

param_grid = [
    {
        "kernel": ["rbf"],
        "C": np.linspace(1e-2, 1e2, num=20),
        "gamma": ["scale"] + list(np.linspace(1e-1, 1e1, num=20)),
    },
    {
        "kernel": ["poly"],
        "C": np.linspace(1e-3, 1e3, 100),
        "degree": [2, 3, 4, 5],
    },
    {
        "kernel": ["sigmoid"],
        "C": np.linspace(1e-2, 1e2, num=20),
        "gamma": ["scale"] + list(np.linspace(1e-1, 1e1, num=20)),
    },
]


cv = GridSearchCV(estimator=clf, param_grid=param_grid, scoring="f1", cv=5, n_jobs=-1)

start_time = time()
cv.fit(X_train, y_train)
end_time = time()
training_time = end_time - start_time

y_pred = cv.predict(X_test)

print(f"F1-score: {100 * f1_score(y_test, y_pred):.2f}%")
print(f"Training time: {training_time} s")

The improvement is tremendous. Let's see what parameters enabled us to get this score

In [ ]:
for param in cv.best_params_:
    print(f"best {param} : {cv.best_params_[param]}")

Now let's see the best validation score for each kernel!

In [ ]:
results_df = pd.DataFrame(cv.cv_results_)

kernels = ["rbf", "poly", "sigmoid"]

for k in kernels:
    score = results_df.loc[results_df["param_kernel"] == k]["mean_test_score"].max()
    print(f"Best validation F1-score for {k} : {100 * score:.2f}%")
    print(results_df.loc[results_df["mean_test_score"] == score].get("params").iloc[0])
    print()

Okay. Let's try to train a model with these parameters and see the test score for each model 

In [ ]:
for k in kernels:
    score = results_df.loc[results_df["param_kernel"] == k]["mean_test_score"].max()
    params = (
        results_df.loc[results_df["mean_test_score"] == score].get("params").iloc[0]
    )

    clf_kernel_svc = SVC(
        cache_size=512,
        class_weight="balanced",
        random_state=0,
        **params,
    )
    start_time = time()
    clf_kernel_svc.fit(X_train, y_train)
    end_time = time()
    training_time = end_time - start_time

    y_pred = clf_kernel_svc.predict(X_test)
    print(f"F1-score for {k} : {100 * f1_score(y_test, y_pred):.2f}%")
    print(f"Training time: {training_time} s")
    print()

During the validation, the rbf kernel performed the best. However, the testing shows that the polynomial kernel worked better

Let's save the best parameters for later

In [ ]:
score = results_df.loc[results_df["param_kernel"] == "poly"]["mean_test_score"].max()
best_kernel_params = (
    results_df.loc[results_df["mean_test_score"] == score].get("params").iloc[0]
)



Now let's try to create a scoring function that will take into account the FN and FP values 

In [ ]:
from typing import Union

from sklearn.metrics import make_scorer


def churn_cost(
    y_true: Union[np.ndarray, pd.Series],
    y_pred: Union[np.ndarray, pd.Series],
    fp_cost: Union[np.ndarray, pd.Series],
    fn_cost: Union[np.ndarray, pd.Series],
) -> float:
    # make sure all data is Numpy arrays
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    fp_cost = np.array(fp_cost)
    fn_cost = np.array(fn_cost)

    # check which rows are for FP and FN
    FP = (y_true == 0) & (y_pred == 1)
    FN = (y_true == 1) & (y_pred == 0)

    # get costs for FP and FN
    fp_real_cost = FP * fp_cost
    fn_real_cost = FN * fn_cost

    # return sum of costs
    return fp_real_cost.sum() + fn_real_cost.sum()


churn_cost_score = make_scorer(
    churn_cost,
    average="macro",
    greater_is_better=False,
)

now let's create two dummy classifiers. One that will always predict the positive class and one that always predicts negative

In [ ]:
def DummyPredict(X, churn):
    return np.full(X.shape[0], churn)


y_pred_p = DummyPredict(X_test, 1)
y_pred_n = DummyPredict(X_test, 0)

print(
    f"F1-score for all positive predictions : {100 * f1_score(y_test, y_pred_p):.2f}%"
)
print(
    f"F1-score for all negative predictions : {100 * f1_score(y_test, y_pred_n):.2f}%"
)

print(
    f"Cost score for all positive predictions : {churn_cost(y_test, y_pred_p, fp_cost_test, fn_cost_test):.2f}"
)
print(
    f"Cost score for all negative predictions : {churn_cost(y_test, y_pred_n, fp_cost_test, fn_cost_test):.2f}"
)

We can see that unfortunately the cost of letting all unsatisfied clients go is much lower than the cost of trying to assume that all clients might leave

How can SVM help us lower that cost?

In [ ]:
lin_clf = LinearSVC(
    loss="hinge",
    max_iter=10000,
    random_state=0,
    class_weight="balanced",
    C=52.48074602497766,
)
lin_clf.fit(X_train, y_train)
y_pred = lin_clf.predict(X_test)
print(f"churn cost: {churn_cost(y_test, y_pred, fp_cost_test, fn_cost_test):.2f}")


clf_kernel_svc = SVC(
    cache_size=512,
    class_weight="balanced",
    random_state=0,
    **best_kernel_params,
)
clf_kernel_svc.fit(X_train, y_train)
y_pred = clf_kernel_svc.predict(X_test)
print(f"churn cost: {churn_cost(y_test, y_pred, fp_cost_test, fn_cost_test):.2f}")

We can lower the cost over 4 times! This means very large fund savings especially for companies that operate on a very large scale

## Kernel approximation - TODO

W przypadku, kiedy zastosowanie kernel SVM wprost byłoby zbyt kosztowne, można zamiast tego przybliżyć cechy kernelowe. Realizują to algorytmy grupy kernel approximation.

Metoda Nyströma to algorytm uniwersalny, który potrafi przybliżyć dowolny kernel. Wykorzystuje do tego truncated SVD, czyli przybliża wprost kernel matrix za pomocą macierzy niższej rangi. Ceną tej prostoty i uniwersalności jest nie najlepsza prędkość działania w porównaniu do bardziej specjalistycznych algorytmów. Proste wyprowadzenie tej metody można znaleźć [tutaj](https://stats.stackexchange.com/questions/261149/nystroem-method-for-kernel-approximation).

`RBFSampler` w Scikit-learn implementuje metodę Random Kitchen Sinks. Ten algorytm za pomocą próbkowania Monte Carlo przybliża coraz lepiej macierz kernela RBF. W bibliotece Scikit-learn-extra zaimplementowane jest rozwinięcie tego algorytmu o nazwie [Fastfood](https://scikit-learn-extra.readthedocs.io/en/stable/generated/sklearn_extra.kernel_approximation.Fastfood.html), który stosuje nieco bardziej zaawansowany algorytm przybliżający, który powinien być jeszcze szybszy i oszczędniejszy pamięciowo.

Hiperparametry tych metod są podobne do zwykłego kernel SVM, ale dodatkowo mamy wymiarowość naszego przybliżenia, czyli `n_components`. Im większe, tym dokładniej przybliżamy, ale też tym więcej mamy cech. Wejściem do metody Nyströma jest macierz cech X, a wyjściem macierz cech kernelowych X mająca rozmiar `n_samples * n_components`. Na takich cechach trenuje się już zwykły liniowy SVM.

1. Zaimplementuj `Pipeline` składający się z przybliżenia kernela oraz liniowego SVM: dla metody Nyströma, oraz dla obu algorytmów dla kernela RBF.
2. Wytrenuj algorytmy z domyślnymi hiperparametrami. Porównaj jakość predykcji oraz szybkość z liniowym SVM i z kernel SVM. Czy warto dokonać takiej aproksymacji?
3. Dokonaj optymalizacji hiperparametrów `C`, `gamma` oraz `n_components`. Czy udało się poprawić wynik względem zwykłego kernel SVM w rozsądnym czasie?